<a href="https://colab.research.google.com/github/mohanaditya910/SalesKen.ai-data_scientist/blob/master/Sentence_similarities/Sentence_similarities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import warnings
warnings.filterwarnings('ignore')


In [0]:

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [0]:
# download the model and return as object ready for use
model_glove_twitter = api.load("glove-twitter-25")

In [89]:
model_glove_twitter.wv.most_similar("baseball",topn=10)


[('soccer', 0.9686072468757629),
 ('basketball', 0.9656766057014465),
 ('football', 0.9512435793876648),
 ('softball', 0.9471019506454468),
 ('championship', 0.9322695136070251),
 ('winning', 0.9318141341209412),
 ('lacrosse', 0.9305177927017212),
 ('volleyball', 0.9303966760635376),
 ('boxing', 0.9276558756828308),
 ('wrestling', 0.9187266230583191)]

In [90]:
model_glove_twitter.wv.similarity(w1="baseball",w2="cricket")

0.8357549

In [91]:
from google.colab import drive
drive.mount ('/content/drive')
%cd /content/drive/My\ Drive/SaleskenProblemSolving-master

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/SaleskenProblemSolving-master


In [0]:
import pandas as pd

In [0]:
df=pd.read_csv("list_of_sentences",header=None,)

In [0]:
corpus=[]
for i in range(len(df)):
  corpus.append(df.iloc[i][0])

In [95]:
corpus

['good morning',
 'how are you doing ?',
 'the weather is awesome today',
 'samsung',
 'good afternoon',
 'baseball is played in the USA',
 'there is a thunderstorm ',
 'are you doing good ?',
 'The polar regions are melting"',
 'apple',
 'nokia',
 'cricket is a fun game',
 'the climate change is a problem']

#new method

In [0]:
import numpy as np

In [0]:
def simple_average(sent):
    sents_emd = []
    for s in sent:
        sent_emd = []
        for w in s:
            if w in model_glove_twitter:
                sent_emd.append(model_glove_twitter[w])
        sent_emd_ar = np.array(sent_emd)
        sum_ = sent_emd_ar.sum(axis=0)
        result = sum_/np.sqrt((sum_**2).sum())
        sents_emd.append(result)
    return sents_emd

In [98]:
sentences=[]
for i in range(len(df)):
  sentences.append(df.iloc[i][0].split(" "))

sentences

[['good', 'morning'],
 ['how', 'are', 'you', 'doing', '?'],
 ['the', 'weather', 'is', 'awesome', 'today'],
 ['samsung'],
 ['good', 'afternoon'],
 ['baseball', 'is', 'played', 'in', 'the', 'USA'],
 ['there', 'is', 'a', 'thunderstorm', ''],
 ['are', 'you', 'doing', 'good', '?'],
 ['The', 'polar', 'regions', 'are', 'melting"'],
 ['apple'],
 ['nokia'],
 ['cricket', 'is', 'a', 'fun', 'game'],
 ['the', 'climate', 'change', 'is', 'a', 'problem']]

#think about stop words..

In [0]:
from scipy.spatial.distance import cosine
from scipy.sparse.linalg import svds


In [100]:
np.shape(model_glove_twitter.syn0)

(1193514, 25)

In [0]:
def tf_idf(sent):
    word_counter = {}
    total_count = 0
    no_of_sentences = 0
    for s in sent:
        for w in s:
            if w in word_counter:
                word_counter[w] = word_counter[w] + 1
            else:
                word_counter[w] = 1
        total_count = total_count + len(s)
        no_of_sentences = no_of_sentences +  1
    sents_emd = []
    for s in sent:
        sent_emd = []
        for word in s:
            tf = word_counter[word]/float(len(s))
            idf = np.log(no_of_sentences/float(1+ word_counter[word]))
            try:
                emd = tf*idf*model_glove_twitter[word]
                sent_emd.append(emd)
            except:
                continue
        sent_emd = np.array(sent_emd)
        sum_ = sent_emd.sum(axis=0)
        result = sum_/np.sqrt((sum_**2).sum())
        sents_emd.append(result)
    return sents_emd

In [0]:
def smooth_inverse_frequency(sent, a=0.001):
    word_counter = {}
    sentences = []
    total_count = 0
    no_of_sentences = 0
    for s in sent:
        for w in s:
            if w in word_counter:
                word_counter[w] = word_counter[w] + 1
            else:
                word_counter[w] = 1
        total_count = total_count + len(s)
        no_of_sentences = no_of_sentences + 1
    sents_emd = []
    for s in sent:
        sent_emd = []
        for word in s:
            if word in model_glove_twitter:
                emd = (a/(a + (word_counter[word]/total_count)))*model_glove_twitter[word]
                sent_emd.append(emd)
        sum_ = np.array(sent_emd).sum(axis=0)
        sentence_emd = sum_/float(no_of_sentences)
        sents_emd.append(sentence_emd)
    [_, _, u]  = np.array(svds(sents_emd, k=1))
    new_sents_emd = []
    for s in sents_emd:
        s = s - s.dot(u*u.transpose())
        new_sents_emd.append(s)
    return new_sents_emd

In [0]:
sifs= smooth_inverse_frequency(sentences, a=0.001)

In [104]:
sifs


[array([-0.00587434,  0.00327272,  0.00203476,  0.00282493,  0.00013173,
         0.00054981,  0.00518015,  0.00323273, -0.00156172,  0.00202933,
        -0.00325844,  0.00088348,  0.00166965,  0.00180771, -0.00025753,
        -0.00357611,  0.00256909, -0.00378086, -0.00303112,  0.0004717 ,
        -0.00033692, -0.0012289 , -0.00219863,  0.00482302,  0.00205984],
       dtype=float32),
 array([ 0.00312952,  0.00143476,  0.00241708,  0.0035904 , -0.00027299,
        -0.00132687,  0.00183525,  0.00227522, -0.00369812,  0.00205476,
        -0.00223232, -0.00127715, -0.00143717, -0.00848624, -0.00484816,
        -0.00211892, -0.00150672, -0.00344552,  0.00068861, -0.00406304,
         0.00699367,  0.00120893, -0.0051396 ,  0.00632835, -0.00299611],
       dtype=float32),
 array([-0.00703394, -0.00556758,  0.0005663 , -0.00521318,  0.00190506,
         0.00261865,  0.00554574, -0.00236263, -0.00135373, -0.00028563,
        -0.00201102,  0.00320638,  0.00325934,  0.00364899,  0.0023943 ,
   

In [105]:
cosine(sifs[10],sifs[9])

0.1273205280303955

In [0]:
##
threshold = 0.2

similarity_scores=10*np.ones((len(sifs),len(sifs)))

for i in range(len(sifs)):
  for j in range(i+1,len(sifs)):
    similarity_scores[i][j]=cosine(sifs[i],sifs[j])

x = np.array(similarity_scores)

update_list = [] #main_list



In [107]:
count=0
while (count < len(sifs)):
  ##IDENTIFYING THE SENTENCES IN THE GROUP...

  grp = []
  i,j = np.unravel_index(np.argmin(x, axis=None), x.shape)
  print ("i,j:{},{}".format(i,j))
  grp.append(i)
  grp.append(j)

  # if there is any value in the ith row which is less than the threshold then grab and append to the list..


  for j1 in range(i+1,len(x)):
    if x[i][j1] <= np.amin(x) + 0.2:
      if j1 not in grp:
        print ("j1:{}".format(j1))
        grp.append(j1)

  for i1 in range(0,j):
    if x[i1][j] <= np.amin(x) + 0.2:
      if i1 not in grp:
        print ("i1:{}".format(i1))
        grp.append(i1)

  #grp
  update_list.append(grp)
  count = count+len(grp)

  '''

  ##REMOVING THE COLUMNS AND ROWS OF THE SENTENCES IN THE GROUP..
  x = np.delete(x,grp,0) #axis=0 rows
  x = np.delete(x,grp,1) #axis=1 columns

  '''
  for l in grp:
    x[l] = 10.0
    x[:,l] = 10.0



  print ("shape_of_x:{}".format(x.shape))

i,j:3,10
j1:9
shape_of_x:(13, 13)
i,j:1,7
shape_of_x:(13, 13)
i,j:0,4
i1:2
shape_of_x:(13, 13)
i,j:5,11
shape_of_x:(13, 13)
i,j:6,12
j1:8
shape_of_x:(13, 13)


In [108]:
update_list

[[3, 10, 9], [1, 7], [0, 4, 2], [5, 11], [6, 12, 8]]

In [0]:
r_final = []
for grp in update_list:
  r_grp = []
  for ind in grp:
    r_grp.append(corpus[ind])

  r_final.append(r_grp)



In [110]:
r_final

[['samsung', 'nokia', 'apple'],
 ['how are you doing ?', 'are you doing good ?'],
 ['good morning', 'good afternoon', 'the weather is awesome today'],
 ['baseball is played in the USA', 'cricket is a fun game'],
 ['there is a thunderstorm ',
  'the climate change is a problem',
  'The polar regions are melting"']]